# Environment Setup

In [1]:
import os
import sys
from collections import Counter

import numpy as np
import open3d as o3d

# Python version
assert sys.version_info >= (3, 8)
# Open3D version
assert o3d.__version__ >= "0.17.0"

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
# Point cloud file I/O
filename = "data/input/iScan-Pcd-1-1.ply"
base_name, extension = os.path.splitext(os.path.basename(filename))

output_filepath = "data/output"
if not os.path.exists(output_filepath):
    os.makedirs(output_filepath)

# 1. Point Cloud Filtering

## Step 1. Down-sampling

In [3]:
def down_sample(input_filename, output_filename):
    # Load the input point cloud file
    pcd = o3d.t.io.read_point_cloud(input_filename)

    # Function uniformly down-samples the point cloud, evenly select 1 point for every k points
    down_sampled_pcd = pcd.uniform_down_sample(every_k_points=10)

    # Save the down-sampled point cloud to an output file
    o3d.t.io.write_point_cloud(output_filename, down_sampled_pcd, write_ascii=False)

In [4]:
# Perform down sampling
down_sampled_filename = os.path.join(output_filepath, base_name + " - downsampled" + extension)
down_sample(filename, down_sampled_filename)

## Step 2. Crop the point cloud
This step is done in `CloudCompare` software manually.

## Step 3. Outlier removal

In [5]:
# Input point cloud file
cropped_filename = os.path.join(output_filepath, base_name + " - cropped" + extension)

In [6]:
def outlier_removal(input_filename, output_filename, visualize=False):
    # Load the input point cloud file
    pcd = o3d.t.io.read_point_cloud(input_filename)

    # Statistical outlier removal
    filtered_pcd, mask = pcd.remove_statistical_outliers(nb_neighbors=20, std_ratio=16.0)

    # Color outliers in red
    outlier = pcd.select_by_mask(mask, invert=True)
    outlier.paint_uniform_color([1.0, 0.0, 0.0])
    print(f"Remove {outlier.point.positions.shape[0]} outliers.")

    # Visualize the outlier
    if visualize:
        inlier = pcd.select_by_mask(mask)
        inlier.paint_uniform_color([0.8, 0.8, 0.8])  # color in grey
        o3d.visualization.draw_geometries([inlier.to_legacy(), outlier.to_legacy()],
                                          window_name="Visualization — {}".format(input_filename),
                                          width=1000, height=800, left=400, top=150)

    # Save the outlier for comparison
    outlier_filename = os.path.join(output_filepath, base_name + " - outlier" + extension)
    o3d.t.io.write_point_cloud(outlier_filename, outlier, write_ascii=False)

    # Save the filtered point cloud to an output file
    o3d.t.io.write_point_cloud(output_filename, filtered_pcd, write_ascii=False)

In [7]:
# Perform outlier removal
filtered_filename = os.path.join(output_filepath, base_name + " - filtered" + extension)
outlier_removal(cropped_filename, filtered_filename, visualize=False)

Remove 151 outliers.


# 2. Point Cloud Segmentation

## Step 1. Mask by intensity

In [8]:
# Input point cloud file
filtered_filename = os.path.join(output_filepath, base_name + " - filtered" + extension)

In [9]:
def mask_intensity(input_filename, output_filename):
    # Load the input point cloud file
    pcd = o3d.t.io.read_point_cloud(input_filename)
    print(pcd.point, '\n')

    # Add mask attribute to acquire low intensity points
    intensity_threshold = 50
    mask = np.where(pcd.point.scalar_intensity.numpy() <= intensity_threshold, 1, 0).astype(np.uint8)
    pcd.point.mask = np.reshape(mask, (len(mask), 1))

    print(pcd.point)

    # Save the masked point cloud to an output file
    o3d.t.io.write_point_cloud(output_filename, pcd, write_ascii=False)

In [10]:
# Perform mask by intensity
masked_filename = os.path.join(output_filepath, base_name + " - masked" + extension)
mask_intensity(filtered_filename, masked_filename)

TensorMap(primary_key="positions") with 2 attributes:
  - positions       : shape={1902497, 3}, dtype=Float64, device=CPU:0 (primary)
  - scalar_intensity: shape={1902497, 1}, dtype=Float32, device=CPU:0
  (Use . to access attributes, e.g., tensor_map.positions) 

TensorMap(primary_key="positions") with 3 attributes:
  - mask            : shape={1902497, 1}, dtype=UInt8, device=CPU:0
  - positions       : shape={1902497, 3}, dtype=Float64, device=CPU:0 (primary)
  - scalar_intensity: shape={1902497, 1}, dtype=Float32, device=CPU:0
  (Use . to access attributes, e.g., tensor_map.positions)


## Step 2. DBSCAN clustering

In [11]:
# Input point cloud file
masked_filename = os.path.join(output_filepath, base_name + " - masked" + extension)

In [12]:
def dbscan_clustering(input_filename, output_filename):
    # Load the input point cloud file
    pcd = o3d.t.io.read_point_cloud(input_filename)

    # Convert form uint8 to bool
    mask = pcd.point.mask.numpy()
    mask = [bool(x) for x in mask]

    # Convert mask to index
    index_of_mask = [i for i, boolean_val in enumerate(mask) if boolean_val]

    # Extract low intensity points, then perform DBSCAN on them
    low_intensity_pcd = pcd.select_by_mask(mask)
    labels = low_intensity_pcd.cluster_dbscan(eps=0.10, min_points=5)
    labels = labels.numpy()

    n_clusters = labels.max() + 1
    print(f"DBSCAN clustering return {n_clusters} clusters.\n")

    # Clusters will be labeled in a way that cluster with the most points is labeled 1
    if n_clusters >= 2:
        counter = Counter(labels)
        print(counter, '\n')

        # Remove noise that be labeled -1
        del counter[-1]

        # Sort by count
        sorted_items = sorted(counter.items(), key=lambda x: x[1], reverse=True)
        rank = [item[0] for item in sorted_items]
        print(rank, '\n')

        # Add cluster attribute to the input point cloud
        cluster = np.zeros(len(mask)).astype(np.int32)
        for index, val in enumerate(labels):
            if val != -1:
                cluster[index_of_mask[index]] = rank.index(val) + 1
        pcd.point.cluster = np.reshape(cluster, (len(cluster), 1))

        print(pcd.point)

        # Save the clustered point cloud to an output file
        o3d.t.io.write_point_cloud(output_filename, pcd, write_ascii=False)

    else:
        print("Warning: DBSCAN clustering return less than 2 clusters.")

In [13]:
clustered_filename = os.path.join(output_filepath, base_name + " - clustered" + extension)
dbscan_clustering(masked_filename, clustered_filename)

DBSCAN clustering return 13 clusters.

Counter({0: 16946, 1: 11024, -1: 583, 6: 57, 8: 51, 11: 50, 5: 36, 12: 25, 2: 9, 3: 8, 4: 7, 9: 6, 7: 5, 10: 5}) 

[0, 1, 6, 8, 11, 5, 12, 2, 3, 4, 9, 7, 10] 

TensorMap(primary_key="positions") with 4 attributes:
  - cluster         : shape={1902497, 1}, dtype=Int32, device=CPU:0
  - mask            : shape={1902497, 1}, dtype=UInt8, device=CPU:0
  - positions       : shape={1902497, 3}, dtype=Float64, device=CPU:0 (primary)
  - scalar_intensity: shape={1902497, 1}, dtype=Float32, device=CPU:0
  (Use . to access attributes, e.g., tensor_map.positions)
